In [1]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras

In [2]:
TEST_DATA_FILE='./data/test'
DATA = './data/data'

In [3]:
test = pd.read_table(
    TEST_DATA_FILE, 
    header = None, 
    names = ['id']
)

In [4]:
data = pd.read_table(
    DATA, 
    delimiter='\t', 
    error_bad_lines=False, 
    header = None, 
    names = ['id', 'comment']
)


In [5]:
test = pd.merge(test, data, how='inner', on=['id'])

In [6]:
list_classes = ['normal', 'insult', 'obscenity', 'threat']

tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(test.comment.values))
list_tokenized_test = tokenizer.texts_to_sequences(test.comment.values)
X_test = pad_sequences(list_tokenized_test, maxlen=100)

In [7]:
model = keras.models.load_model(os.path.join(os.getcwd()))

In [8]:
predictions = model.predict(X_test, batch_size=1024, verbose=1)

98/98 [==============================] - 109s 1s/step


In [15]:
test = test.drop(columns=['comment'])

In [16]:
result = pd.concat([test, pd.DataFrame(data=predictions, columns=list_classes)], axis=1)

In [26]:
result = result[['id', 'normal', 'insult','obscenity', 'threat']]
result

,id,normal,insult,obscenity,threat
0,167315,0.995909,0.003261,0.000197,0.000033
1,224546,0.925806,0.059715,0.004366,0.010631
2,241309,0.977234,0.019940,0.002071,0.000470
3,31170,0.988559,0.009253,0.000191,0.000331
4,173358,0.999023,0.000893,0.000003,0.000006
...,...,...,...,...,...
99510,192320,0.928974,0.069144,0.001157,0.003284
99511,6646,0.997226,0.002566,0.000070,0.000047
99512,215218,0.966785,0.039936,0.000104,0.000304
99513,139806,0.981752,0.016142,0.000797,0.000558


In [25]:
predictions = result.to_numpy()
np.savetxt("result.txt", predictions, fmt="%.6g")

In [27]:
predictions

array([[1.67315000e+05, 9.95909035e-01, 3.26138735e-03, 1.97052956e-04,
        3.27495909e-05],
       [2.24546000e+05, 9.25806284e-01, 5.97152114e-02, 4.36633825e-03,
        1.06309950e-02],
       [2.41309000e+05, 9.77233946e-01, 1.99396610e-02, 2.07135081e-03,
        4.69744205e-04],
       ...,
       [2.15218000e+05, 9.66784656e-01, 3.99363935e-02, 1.04139443e-04,
        3.04073095e-04],
       [1.39806000e+05, 9.81752098e-01, 1.61421299e-02, 7.96824694e-04,
        5.58197498e-04],
       [9.90520000e+04, 9.93769109e-01, 6.80552609e-03, 5.21859729e-05,
        5.53345781e-05]])